<a href="https://colab.research.google.com/github/ganeshsprofessional/sdc-GenAI/blob/main/langchain_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio langchain requests --quiet


In [ ]:
import os
import gradio as gr
import requests
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.llms.base import LLM

# 🔑 Your DeepAI API Key
DEEPAI_API_KEY = "3e3ff45f-23c7-4571-beab-9099ee9ff942"  # 👈 Replace this

# 🔌 Create a custom LLM class to wrap DeepAI
class DeepAILLM(LLM):
    def _call(self, prompt: str, stop=None):
        try:
            response = requests.post(
                "https://api.deepai.org/api/text-generator",
                data={"text": prompt},
                headers={"api-key": DEEPAI_API_KEY}
            )
            result = response.json()
            output = result.get("output")
            if not output:
                return "⚠️ No response received from DeepAI."
            return output.strip()
        except Exception as e:
            return f"❌ Error: {str(e)}"

    @property
    def _llm_type(self):
        return "deepai"

# 🧠 Setup LangChain with memory
llm = DeepAILLM()
memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm, memory=memory)

# 🗣️ Chat handler
def chat_with_bot(message, history):
    response = conversation.predict(input=message)
    history.append((message, response))
    return history, history, ""  # "" clears the textbox

# 🧪 Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 💬 Chat with AI (LangChain + DeepAI)")

    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Type your message...", label="Your Message")
    clear = gr.Button("Clear Chat")
    state = gr.State([])

    msg.submit(chat_with_bot, inputs=[msg, state], outputs=[chatbot, state, msg])
    clear.click(lambda: ([], [], ""), None, [chatbot, state, msg])

demo.launch(share=True)

<ipython-input-4-63bbc70ee128>:47: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5bfa60696188cd1c29.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
